# Notebook Imports

In [1]:
import numpy as np
import pandas as pd

# Constants

In [2]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'
VOCAB_SIZE = 2500

# Read and Load Features from .txt Files into NumPy Array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [4]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [5]:
sparse_train_data[:5]

array([[ 0,  0,  1,  2],
       [ 0,  7,  1,  1],
       [ 0, 16,  1,  1],
       [ 0, 18,  1,  1],
       [ 0, 24,  1,  1]])

In [6]:
sparse_train_data[-5:]

array([[5795, 1834,    0,    1],
       [5795, 1905,    0,    2],
       [5795, 2105,    0,    1],
       [5795, 2257,    0,    1],
       [5795, 2463,    0,    1]])

In [7]:
print('Nr of rows in training file', sparse_train_data.shape[0])
print('Nr of rows in test file', sparse_test_data.shape[0])

Nr of rows in training file 265577
Nr of rows in test file 110578


In [8]:
print('Nr of emails in training file', np.unique(sparse_train_data[:,0]).size)

Nr of emails in training file 4014


In [9]:
print('Nr of emails in testing file', np.unique(sparse_test_data[:,0]).size)

Nr of emails in testing file 1723


## How to create an Empty DataFrame

In [10]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [11]:
len(column_names)

2502

In [12]:
index_names = np.unique(sparse_train_data[:,0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [13]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True) #Fill empty dataframe with 0

In [14]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Create a Full Matrix from a Sparse Matrix

In [15]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    '''
    Form a full matrix from a sparse matrix. Return a pandas dataframe.
    Keyword arguments:
    sparse_matrix --numpy array
    nr_words -- size of the vocabulary. Total number of tokens.
    doc_idx -- position of the document id in the sparse martrix. Default: 1st column
    word_idx --position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column
    '''
    
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:,0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]): # 0 -> nr of rows in sparse matrix
        
        # Store all the values in a particular row in four separate variables
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        # Selecting single cell and setting its values
        # Grabing data from sparse matrix and populating it in empty dataFrame
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
        
    #
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix

In [23]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

Wall time: 26.4 s


In [24]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,2,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,0,1,2,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2,0,2,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,3,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,2,4,2,3,1,...,0,0,0,0,0,0,0,0,0,0


In [25]:
full_train_data.tail()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5789,0,0,0,0,0,0,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,2,2,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
5791,0,2,0,1,1,0,1,2,0,1,...,0,0,0,0,0,0,0,0,0,0
5794,0,2,2,1,0,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
5795,0,1,1,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Training the Naive Bayes Model

## Calculating the Probability of Spam

In [26]:
# Challenge: Calculate the probability of spam - the percent of spam messages in the training dataset
# Store this value in a variable called prob_spam

In [30]:
# Total nr of messages
full_train_data.CATEGORY.size

4014

In [31]:
# Nr of spam messages
full_train_data.CATEGORY.sum()  #1-spam; in this way only spam would be sum

1249

In [33]:
# Prob of spam
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('Probability of spam is', prob_spam)

Probability of spam is 0.3111609367214748


## Total Number of Words / Tokens

In [34]:
# Creating a subset by including all rows and excluding CATEGORY column
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,2,4,2,3,1,1,...,0,0,0,0,0,0,0,0,0,0


In [37]:
# This array would hold every email tokens
email_lengths = full_train_features.sum(axis=1)  #axis=1; to sum columns of every row
email_lengths.shape

(4014,)

In [38]:
email_lengths[:5]

DOC_ID
0     50
1     76
2     87
3     76
4    137
dtype: int64

In [41]:
total_wc = email_lengths.sum()
print('Total nr of tokens of all emails:', total_wc)

Total nr of tokens of all emails: 444964


## Number of Tokens in Spam and Ham Emails

**Challenge** Create a subset of the email_lengths series that only contains the spam messages. Call the spam_lenghts. Then count the total number of words that occurs in spam emails.

Do the same for the non-spam emails. Create a subset called ham_lengths. Then count the total number of words that occur in ham emails.

In [45]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths.shape

(1249,)

In [46]:
# Sum of spam emails tokens
spam_wc = spam_lengths.sum()
spam_wc

193997

In [52]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape

(2765,)

In [53]:
# Quick check to verify the results
email_lengths.shape[0] - spam_lengths.shape[0] - ham_lengths.shape[0]

0

In [54]:
# Sum of ham emails tokens
nonspam_wc = ham_lengths.sum()
nonspam_wc

250967

In [56]:
# Challenge: Can you figure out if spam emails or non-spam emails tend to be longer? Which category
# contains more tokens? Take a guess and then verify your intuition using the dataset

In [61]:
print('Average nr of words in spam emails {:.2f}'.format(spam_wc / spam_lengths.shape[0]))
print('Average nr of words in non-spam emails {:.3f}'.format(nonspam_wc / ham_lengths.shape[0]))

Average nr of words in spam emails 155.32
Average nr of words in non-spam emails 90.766
